In [5]:
import numpy as np
import pandas as pd
import requests
import json
import os

pd.set_option('display.max_columns', 100)


In [6]:

BTCUSDT_DAY_ID = "81d625dd-f73f-4829-a2ff-1d795fbc5b3a"
BTCUSDT_HOUR_ID = "7aa396ec-8c6a-44fd-89f2-12e7f998f883"
BTCUSDT_MINUTE_ID = "31deee63-e4bb-45ad-b020-8759b5c69e0f"
ETHUSDT_DAY_ID = "67da12e9-6fd9-4401-b16a-1c2dd589ff01"
ETHUSDT_HOUR_ID = "35515212-5467-45b4-b2e5-3ae048e482e7"
ETHUSDT_MINUTE_ID = "38d4f96b-26b7-420a-a34f-912f5ad23eaf"


ASSET_ID = BTCUSDT_DAY_ID

base_url = "http://172.24.100.128:5000/"

PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [7]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end},proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/",proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            pivoted_df = pd.pivot_table(df, index='price_id', columns='name', values='value', aggfunc=sum, fill_value=0)

            pivoted_df = pivoted_df.replace(0, np.nan)

            return pivoted_df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url, asset_id, unix_time_start, unix_time_end):

    df = api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end)

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df

    return df_merge



In [8]:
df = get_indicators_and_merge(base_url, ASSET_ID, 1648749254, 1680982906)


In [9]:
df


,date_time_utc,num_trades,taker_base_vol,open_price,taker_quote_vol,close_price,low_price,id,high_price,volume,unix_time,qav,NaturalGas,Russell2000,accdist,adx,adx_neg,adx_pos,ao,aroon_down,aroon_up,atr,bb_pb,bb_wb,bbh,bbhi,bbl,bbli,bbmavg,cci,cmf,cocoa,coffee,corn,cotton,dchb,dclb,dcmb,dcpb,dcwb,dowJones,dpo,ema,emv,fi,gold,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,kama,kchb,kchbi,kclb,kclbi,kcmb,kcpb,kcwb,kst,kst_sig,macd,macd_diff,macd_signal,mfi,mi,nasdaq,nvi,oat,obv,ppo,pvo,rice,roc,rsi,silver,sma,smaemv,soybean,spy500,stc,stoch,stochrsi,sugar,trix,tsi,ulcer,uo,usBond,usBond10y,usBond2y,usBond5y,vi_neg,vi_pos,vpt,vwap,vwheat,wma,wri
0,2022-04-01T00:00:00+00:00,1372216,28041.37621,45510.35,1.274360e+09,46283.49,44200.00,2d55e307-7763-4440-abe5-8be4e137807f,46720.09,56271.06474,1648771200,2.556533e+09,5.720,2088.300049,5.566593e+05,23.717550,19.058903,26.656030,4876.513382,24,88,1881.544108,0.776457,26.135759,48805.371970,NaN,37523.959030,NaN,43164.6655,72.348226,0.207081,2599.0,228.399994,735.00,134.550003,48189.84,37555.00,44970.655,0.703908,14.120928,34718.0,-2162.4155,45065.265345,-4.209572e+09,1.296544e+07,1919.099976,44023.7850,41256.060,42672.420,45375.150,45724.076037,44758.955500,1.0,41317.519500,NaN,43038.237500,1.442994,7.996229,132.739474,78.076175,1688.631021,347.472727,1341.158294,64.924683,23.187088,14863.75,1506.119284,740.00,1.072698e+03,3.892951,-0.769070,1600.5,12.169470,62.364606,24.642000,44942.635000,1.008147e+09,1582.75,4539.25,99.999192,75.312423,0.568070,19.370001,0.467759,24.189059,1.602908,56.807108,149.81250,122.234375,105.683594,114.023438,0.804169,1.181123,-653.885154,44526.431167,1013.00,46306.500889,-24.687577
1,2022-04-02T00:00:00+00:00,1056716,18562.93079,46283.49,8.621978e+08,45811.00,45620.00,eb48a66c-f522-4d7f-a9ce-f4fda222394c,47213.00,37073.53582,1648857600,1.721691e+09,NaN,NaN,5.284760e+05,23.461928,17.893557,26.918108,4439.513088,20,84,1860.933815,0.716925,23.751328,48740.141557,NaN,38392.555443,NaN,43566.3485,77.051131,0.202929,NaN,NaN,NaN,NaN,48189.84,37555.00,45375.150,0.577424,12.267826,NaN,-1202.2185,45179.993754,4.109759e+09,8.610821e+06,NaN,44378.4200,41256.060,42872.420,45884.420,45731.530834,45151.666833,1.0,41724.079833,NaN,43437.873333,1.192361,7.890780,141.339756,92.114987,1623.036046,225.502202,1397.533845,65.541477,23.358602,NaN,1490.743903,NaN,-3.600084e+04,3.726238,-2.381502,NaN,11.728015,60.014501,NaN,45343.364167,1.142424e+09,NaN,NaN,99.999596,69.193592,0.436733,NaN,0.491226,23.712824,1.845630,52.159201,NaN,NaN,NaN,NaN,0.791964,1.178112,577.488813,44742.601725,NaN,46289.534667,-30.806408
2,2022-04-03T00:00:00+00:00,966563,16740.18067,45810.99,7.776160e+08,46407.35,45530.92,c6f611e4-3c20-4c09-8cfe-bf75a05c38e5,47444.11,33394.67794,1648944000,1.551000e+09,NaN,NaN,5.256774e+05,23.343634,16.582186,25.830650,4066.122029,16,80,1864.666399,0.750950,22.729293,48892.585017,NaN,38913.709983,NaN,43903.1475,76.249053,0.162915,NaN,NaN,NaN,NaN,48189.84,38098.33,45884.420,0.613413,9.995548,NaN,-1020.3875,45368.817791,4.068468e+08,1.022569e+07,NaN,44501.6625,41256.060,42872.420,46130.905,45779.452529,45497.868500,1.0,42118.174500,NaN,43808.021500,1.269102,7.714783,147.033739,105.058442,1600.719914,162.548856,1438.171059,70.387633,23.400394,NaN,1510.149835,NaN,-2.606160e+03,3.657275,-4.331102,NaN,9.543970,61.962780,NaN,45680.299167,1.283176e+09,NaN,NaN,99.999798,76.916432,0.545613,NaN,0.508569,23.843281,1.840603,47.018935,NaN,NaN,NaN,NaN,0.782455,1.195235,56.249900,44992.945403,NaN,46351.404222,-23.083568
3,2022-04-04T00:00:00+00:00,1171998,21913.04154,46407.36,1.008164e+09,46580.51,45118.00,24879eda-a5d5-4d6e-b2b3-bc92b9304b1b,46890.71,44641.87514,1649030400,2.053435e+09,5.712,2094.600098,5.546958e+05,22.897849,17.039512,24.070392,3789.933000,12,76,1858.098085,0.749225,20.959024,48907.243276,NaN,39629.069724,NaN,44268.1565,62.535362,0.178938,2562.0,230.600006,750.50,137.940002,48189.84,38828.48,46130.905,0.609184,8.883206,34829.0,-276.6965,45555.231977,-1.91